In [1]:
import numpy as np
import torch as th
from torch import nn as nn
import torch.nn.functional as F
from torch import tensor
    

In [2]:
model_path = 'tmp/best_model.zip'

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(7, 32, kernel_size=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=1)
        self.fc3 = nn.Linear(2112, 512)
        self.shared1 = nn.Linear(512, 64)
        self.policy1 = nn.Linear(64, 32)
        self.policy2 = nn.Linear(32, 16)
        self.action = nn.Linear(16, 4)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = nn.Flatten()(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.shared1(x))
        x = F.relu(self.policy1(x))
        x = F.relu(self.policy2(x))
        x = self.action(x)
        x = x.argmax()
        return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.shared1 = th.nn.Sequential()
        self.policy1 = nn.Linear(231, 64)
        self.policy2 = nn.Linear(64, 64)
        self.action = nn.Linear(64, 4)

    def forward(self, x):
        x = nn.Flatten(start_dim=1, end_dim=-1)(x)
        x = F.tanh(self.policy1(x))
        x = F.tanh(self.policy2(x))
        x = self.action(x)
        x = x.argmax()
        return x


mlp_extractor.policy_net.0.weight', 'mlp_extractor.policy_net.0.bias', 'mlp_extractor.policy_net.2.weight', 'mlp_extractor.policy_net.2.bias', 'mlp_extractor.value_net.0.weight', 'mlp_extractor.value_net.0.bias', 'mlp_extractor.value_net.2.weight', 'mlp_extractor.value_net.2.bias', 'action_net.weight', 'action_net.bias', 'value_net.weight', 'value_net.bias'

ctorCriticPolicy(  
  (features_extractor): FlattenExtractor(  
    (flatten): Flatten(start_dim=1, end_dim=-1)  
  )  
  (mlp_extractor): MlpExtractor(  
    (shared_net): Sequential()  
    (policy_net): Sequential(  
      (0): Linear(in_features=231, out_features=64, bias=True)  
      (1): Tanh()  
      (2): Linear(in_features=64, out_features=64, bias=True)  
      (3): Tanh()  
    )  
    (value_net): Sequential(  
      (0): Linear(in_features=231, out_features=64, bias=True)  
      (1): Tanh() 
      (2): Linear(in_features=64, out_features=64, bias=True)  
      (3): Tanh()  
    )  
  )  
  (action_net): Linear(in_features=64, out_features=4, bias=True)  
  (value_net): Linear(in_features=64, out_features=1, bias=True)  
)  

In [4]:
model = Net()

In [5]:
model = th.load('tmp/best_pytorch_model')

In [7]:
model.keys()

odict_keys(['actor.latent_pi.0.weight', 'actor.latent_pi.0.bias', 'actor.latent_pi.2.weight', 'actor.latent_pi.2.bias', 'actor.mu.weight', 'actor.mu.bias', 'actor.log_std.weight', 'actor.log_std.bias', 'critic.qf0.0.weight', 'critic.qf0.0.bias', 'critic.qf0.2.weight', 'critic.qf0.2.bias', 'critic.qf0.4.weight', 'critic.qf0.4.bias', 'critic.qf1.0.weight', 'critic.qf1.0.bias', 'critic.qf1.2.weight', 'critic.qf1.2.bias', 'critic.qf1.4.weight', 'critic.qf1.4.bias', 'critic_target.qf0.0.weight', 'critic_target.qf0.0.bias', 'critic_target.qf0.2.weight', 'critic_target.qf0.2.bias', 'critic_target.qf0.4.weight', 'critic_target.qf0.4.bias', 'critic_target.qf1.0.weight', 'critic_target.qf1.0.bias', 'critic_target.qf1.2.weight', 'critic_target.qf1.2.bias', 'critic_target.qf1.4.weight', 'critic_target.qf1.4.bias'])

In [7]:
def get_geese_observation(state):
    """
    Given a particular geese, does some processing and returns a geese specific observation. 
    Unfortunately specific to the geese environment for now.
    Encoding as follows: 
    2: enemy snake head
    1: enemy snake body
    11: own head
    12: own body
    100: food
    """

    if type(state) is list:
        state = state[0].observation
    else:

        state = Observation(state)

    
    agent = state.index
    rows = 7
    columns = 11
    game_board_self = np.zeros(rows*columns, None)
    game_board_enemy = np.zeros(rows*columns, None)
    game_board_food = np.zeros(rows*columns, None)


    for i, geese in enumerate(state.geese):
        identify=0
        if i==agent:
            identify=100
            for j, cell in enumerate(geese):
                if j == 0:
                    game_board_self[cell] = identify+1
                else:
                    game_board_self[cell] = identify+2
        else:
            identify=-100
            for j, cell in enumerate(geese):
                if j == 0:
                    game_board_enemy[cell] = identify+1
                else:
                    game_board_enemy[cell] = identify+2

    for food in state.food:
        game_board_food[food] = 1000
    game_board_self = game_board_self.reshape([rows, columns])
    game_board_enemy = game_board_enemy.reshape([rows, columns])
    game_board_food = game_board_food.reshape([rows, columns])

    head = get_geese_coord(game_board_self)[0]


    game_board_self = np.roll(game_board_self, 5-head[1], axis=1)
    game_board_self = np.roll(game_board_self, 3-head[0], axis=0)
    game_board_enemy = np.roll(game_board_enemy, 5-head[1], axis=1)
    game_board_enemy = np.roll(game_board_enemy, 3-head[0], axis=0)
    game_board_food = np.roll(game_board_food, 5-head[1], axis=1)
    game_board_food = np.roll(game_board_food, 3-head[0], axis=0)

    #game_board = game_board.reshape((game_board.shape[0], game_board.shape[1], 1))
    game_board = np.dstack((game_board_self, game_board_enemy, game_board_food))
    return game_board

def get_geese_coord(board):
    return get_coord_from_np_grid(board, 101)

def get_food_coord(board):
    return get_coord_from_np_grid(board, 1000)

def get_enemy_geese_head_coord(board):
    return get_coord_from_np_grid(board, -99)


def get_coord_from_np_grid(grid, value):
    coords = []
    for i in range(0, len(np.where(grid==value)[0])):
        coords.append((np.where(grid==value)[0][i], np.where(grid==value)[1][i]))
    return coords


In [8]:
'''
state_dict = {
    'conv1.weight': state_dict['features_extractor.conv1.weight'],
    'conv1.bias': state_dict['features_extractor.conv1.bias'],
    'conv2.weight': state_dict['features_extractor.conv2.weight'],
    'conv2.bias': state_dict['features_extractor.conv2.bias'],
    'fc3.weight': state_dict['features_extractor.fc3.weight'],
    'fc3.bias': state_dict['features_extractor.fc3.bias'],

    'shared1.weight': state_dict['mlp_extractor.shared_net.0.weight'],
    'shared1.bias': state_dict['mlp_extractor.shared_net.0.bias'],

    'policy1.weight': state_dict['mlp_extractor.policy_net.0.weight'],
    'policy1.bias': state_dict['mlp_extractor.policy_net.0.bias'],
    'policy2.weight': state_dict['mlp_extractor.policy_net.2.weight'],
    'policy2.bias': state_dict['mlp_extractor.policy_net.2.bias'],

    'action.weight': state_dict['action_net.weight'],
    'action.bias': state_dict['action_net.bias'],
}
'''

"\nstate_dict = {\n    'conv1.weight': state_dict['features_extractor.conv1.weight'],\n    'conv1.bias': state_dict['features_extractor.conv1.bias'],\n    'conv2.weight': state_dict['features_extractor.conv2.weight'],\n    'conv2.bias': state_dict['features_extractor.conv2.bias'],\n    'fc3.weight': state_dict['features_extractor.fc3.weight'],\n    'fc3.bias': state_dict['features_extractor.fc3.bias'],\n\n    'shared1.weight': state_dict['mlp_extractor.shared_net.0.weight'],\n    'shared1.bias': state_dict['mlp_extractor.shared_net.0.bias'],\n\n    'policy1.weight': state_dict['mlp_extractor.policy_net.0.weight'],\n    'policy1.bias': state_dict['mlp_extractor.policy_net.0.bias'],\n    'policy2.weight': state_dict['mlp_extractor.policy_net.2.weight'],\n    'policy2.bias': state_dict['mlp_extractor.policy_net.2.bias'],\n\n    'action.weight': state_dict['action_net.weight'],\n    'action.bias': state_dict['action_net.bias'],\n}\n"

In [9]:
def agent(obs, config):
    model = ConvNet()
    model = model.float()
    state_dict = th.load('tmp/best_pytorch_model')
    state_dict = {
        'conv1.weight': state_dict['features_extractor.conv1.weight'],
        'conv1.bias': state_dict['features_extractor.conv1.bias'],
        'conv2.weight': state_dict['features_extractor.conv2.weight'],
        'conv2.bias': state_dict['features_extractor.conv2.bias'],
        'fc3.weight': state_dict['features_extractor.fc3.weight'],
        'fc3.bias': state_dict['features_extractor.fc3.bias'],

        'shared1.weight': state_dict['mlp_extractor.shared_net.0.weight'],
        'shared1.bias': state_dict['mlp_extractor.shared_net.0.bias'],

        'policy1.weight': state_dict['mlp_extractor.policy_net.0.weight'],
        'policy1.bias': state_dict['mlp_extractor.policy_net.0.bias'],
        'policy2.weight': state_dict['mlp_extractor.policy_net.2.weight'],
        'policy2.bias': state_dict['mlp_extractor.policy_net.2.bias'],

        'action.weight': state_dict['action_net.weight'],
        'action.bias': state_dict['action_net.bias'],
    }
    model.load_state_dict(state_dict)
    model.eval()
    obs = tensor(get_geese_observation(obs)).reshape(1, 7, 11, 3).float()
    action = model(obs)
    print(action)
    return int(action)

In [10]:
from GeeseGymWrapper import HungryGeeseKaggle

Loading environment football failed: No module named 'gfootball'


In [11]:
env = HungryGeeseKaggle()

In [12]:
env.env.reset(num_agents=4)

[{'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'geese': [[10], [72], [4], [70]],
   'food': [29, 60],
   'index': 0},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 1},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 2},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 3},
  'status': 'ACTIVE'}]

In [13]:
obs = env.env.state

In [14]:
obs

[{'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'geese': [[10], [72], [4], [70]],
   'food': [29, 60],
   'index': 0},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 1},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 2},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 3},
  'status': 'ACTIVE'}]

In [15]:
config = {}

In [16]:
agent(obs, config)

RuntimeError: Error(s) in loading state_dict for ConvNet:
	size mismatch for conv1.weight: copying a param with shape torch.Size([32, 3, 1, 1]) from checkpoint, the shape in current model is torch.Size([32, 7, 1, 1]).
	size mismatch for fc3.weight: copying a param with shape torch.Size([512, 4928]) from checkpoint, the shape in current model is torch.Size([512, 2112]).
	size mismatch for shared1.weight: copying a param with shape torch.Size([512, 512]) from checkpoint, the shape in current model is torch.Size([64, 512]).
	size mismatch for shared1.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for policy1.weight: copying a param with shape torch.Size([256, 512]) from checkpoint, the shape in current model is torch.Size([32, 64]).
	size mismatch for policy1.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for policy2.weight

In [ ]:
import os
os.listdir()

In [ ]:
path = os.getcwd()

In [ ]:
path

In [ ]:
# Check that if I load the same model from stable baselines I get the same output
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor

log_dir = "tmp/"
env = HungryGeeseKaggle()
env = Monitor(env, log_dir)
model = PPO('MlpPolicy', env, verbose=1)



In [ ]:
model = model.load('best_model.zip')

In [ ]:
obs

In [ ]:
model.policy.predict(get_geese_observation(obs), deterministic=True)

In [ ]:
model.policy.state_dict()